In [1]:
import numpy as np

<h3>Loading regression data</h3>

In [2]:
import pandas as pd
data = pd.read_csv('student.csv').drop('Extracurricular Activities',axis=1)
X = data.drop('Performance Index',axis=1).values
y = data['Performance Index'].values

In [3]:
X[:3],y[:3]

(array([[ 7, 99,  9,  1],
        [ 4, 82,  4,  2],
        [ 8, 51,  7,  2]], dtype=int64),
 array([91., 65., 45.]))

<h4>Scaling X's columns</h4>

In [4]:
for i in range(X.shape[1]):
    col_min = X[:,i].min()
    col_max = X[:,i].max()
    X[:,i] = (X[:,i]-col_min)/(col_max-col_min)
X = np.round(X,3)

<h3>Scaling y</h3>

In [5]:
y = (y-y.min())/(y.max()-y.min())

In [6]:
X[:3],y[:3]

(array([[0, 1, 1, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=int64),
 array([0.9       , 0.61111111, 0.38888889]))

In [7]:
train_index = int(0.65*len(X))
val_index = int(0.75*len(X))
X_train,X_val,X_test = X[0:train_index],X[train_index:val_index],X[val_index:]
y_train,y_val,y_test = y[0:train_index],y[train_index:val_index],y[val_index:]

In [8]:
class LinearRegressor:
    def __init__(self,lr=0.01,penalty='l2',penalty_alpha=0.01,l1_ratio=0.5,force_positive=False):
        self.loss_func = None
        self.learning_rate = lr
        self.l1_ratio = l1_ratio
        self.penalty = penalty
        self.penalty_alpha = penalty_alpha 
        self.w = None
        self.force_positive = force_positive
        self.penalty_func = self.__None_penalty
        self.__init_penalty_func()

    def __init_penalty_func(self):
        if self.penalty == 'l2':
            self.penalty_func = self.__l2_penalty
        elif self.penalty == 'l1':
            self.penalty_func = self.__l1_penalty
        elif self.penalty == 'elastic':
            self.penalty_func = self.__elastic_penalty
        
    def __mean_squared_error(self,y_true,y_pred):
        return sum((y_true-y_pred)**2)/len(y_true)

    def __l1_penalty(self):
        return np.sign(self.w)*self.penalty_alpha

    def __l2_penalty(self):
        return self.w*self.penalty_alpha

    def __None_penalty(self):
        return 0

    def __elastic_penalty(self):
        return ((1-self.l1_ratio) * self.__l2_penalty()) + (self.l1_ratio * self.__l1_penalty())

    def __backpropagation(self,X_train,y_true,y_pred):        
        gradient = np.dot(X_train.transpose(),(2/len(y_true))*(y_pred-y_true))
        gradient += self.penalty_func()
        self.w = self.w - self.learning_rate*gradient
        if self.force_positive:
            self.w = self.w*(self.w > 0)

    def __epoch(self,X_train,y_train):
        y_pred = np.dot(X_train,self.w)
        self.__backpropagation(X_train,y_train,y_pred)

    def __init_weights(self,X):
        self.w = np.zeros((X.shape[1]+1, 1)) 

    def __return_expanded_X(self,X):
        ones_column = np.ones((X.shape[0], 1))
        return np.hstack((X, ones_column))  
    
    def __return_expanded_y(self,y):
        if len(y.shape) == 2:
            return y
        return np.expand_dims(y,axis=1)
        
    def fit(self,X_train_,y_train_,X_val_=None,y_val_=None,epochs=100,patience = 10):
        if self.w is None:
            self.__init_weights(X_train_)
        
        X_train = self.__return_expanded_X(X_train_)
        y_train = self.__return_expanded_y(y_train_)
        
        if (X_val_ is not None) and (y_val_ is not None):
            X_val = self.__return_expanded_X(X_val_)
            y_val = self.__return_expanded_y(y_val_)
            
        epochs_no_improvement = 0
        best_validation_error = np.inf
        best_weights = self.w
        for epoch in range(epochs):
            self.__epoch(X_train,y_train)
            
            #Evaluate model
            if X_val_ is not None:
                val_error = self.__mean_squared_error(y_val,np.dot(X_val,self.w))
                if val_error < best_validation_error:
                    best_validation_error = val_error
                    best_weights = self.w.copy()
                    epochs_no_improvement = 0
                else:
                    epochs_no_improvement += 1
                    if epochs_no_improvement > patience:
                        self.w = best_weights
                        break
        if X_val_ is not None:
            print("ran for ",epoch," epochs")


    def predict(self,X):
        return np.dot(self.__return_expanded_X(X),self.w)

<h4>Error metric</h4>

In [9]:
def rmse(y_true,y_pred):
    return np.sqrt(sum((y_true-y_pred)**2)/len(y_true))

In [10]:
lr = LinearRegressor(penalty='elastic',lr=0.01,penalty_alpha=0.01,force_positive=False)
lr.fit(X_train,y_train,X_val,y_val,epochs=1_000,patience=100)
y_pred = lr.predict(X_test).reshape((len(y_test),))
print("Our model's error is : " , np.round(rmse(y_test,y_pred),4))

ran for  999  epochs
Our model's error is :  0.2081


<h3>Now we compare to Sklearn's ElasticNet</h3>

In [11]:
from sklearn.linear_model import ElasticNet

In [12]:
sk_lr = ElasticNet(tol=0.0001,max_iter=1_000,alpha=0.01,positive=False)
sk_lr.fit(X_train,y_train)
sk_y_pred = sk_lr.predict(X_test).reshape((len(y_test),))
print("Sklearn's ElasticNet model's error is : " , np.round(rmse(y_test,sk_y_pred),4))

Sklearn's ElasticNet model's error is :  0.2094


<h3>Not bad!</h3>